In [1]:
import torch
import torch.nn as nn
import torchvision as tv
import torchvision.transforms as transforms
from torchvision import datasets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import random
import time
import os

In [2]:
SAMPLE_SIZE = 5000
NUM_LABELS = 10

In [3]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Load data
dataset = datasets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())

# Normalize images
all_images = torch.stack([dataset[i][0] for i in range(len(dataset))])
all_labels = torch.tensor([dataset[i][1] for i in range(len(dataset))])

cifar10_mean = np.array(all_images.mean(dim=(0, 2, 3)))
cifar10_std = np.array(all_images.std(dim=(0, 2, 3)))

mean_tensor = torch.tensor(cifar10_mean).view(1, 3, 1, 1)
std_tensor = torch.tensor(cifar10_std).view(1, 3, 1, 1)

normalized_images = (all_images - mean_tensor) / std_tensor

# Subset first 5k samples
images = normalized_images[:SAMPLE_SIZE]  # Use normalized images
subset_labels = all_labels[:SAMPLE_SIZE]

# Convert labels to one-hot encoding for MSE loss (CIFAR-10 has 10 classes)
labels = torch.zeros(subset_labels.size(0), NUM_LABELS, device=subset_labels.device)
labels.scatter_(1, subset_labels.unsqueeze(1), 1)

Files already downloaded and verified


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [5]:
class FullyConnectedNet(nn.Module):
    def __init__(self, input_size, hidden_layer_1_size, hidden_layer_2_size, num_labels):
        super(FullyConnectedNet, self).__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, hidden_layer_1_size, bias=True),
            nn.Tanh(),
            nn.Linear(hidden_layer_1_size, hidden_layer_2_size, bias=True),
            nn.Tanh(),
            nn.Linear(hidden_layer_2_size, num_labels, bias=True)
        )
    
    def forward(self, x):
        return self.network(x)

In [6]:
def compute_sharpness(model, criterion, images, labels,
                      iters: int = 20, tol: float = 1e-3,
                      subsample: int | None = 1024, damping: float = 0.0) -> float:
    """
    Estimates λ_max(H) (sharpness) of the loss at current model parameters via
    power iteration with Hessian–vector products (Pearlmutter trick).

    Args
    ----
    model      : nn.Module (params require_grad=True)
    criterion  : callable(logits, targets) -> scalar loss (mean reduction)
    images     : tensor [N, ...] on correct device
    labels     : tensor [N] on correct device
    iters      : power-iteration steps (15–25 typical)
    tol        : relative convergence tolerance
    subsample  : if not None, randomly sample this many examples for speed
    damping    : computes eigenvalues of (H + damping * I)

    Returns
    -------
    float: estimated largest eigenvalue (sharpness)
    """
    was_training = model.training
    model.eval()  # stabilize stats (esp. BN/Dropout)

    # ---- choose subset (for speed/memory) ----
    if subsample is not None and images.size(0) > subsample:
        idx = torch.randperm(images.size(0), device=images.device)[:subsample]
        xb, yb = images[idx], labels[idx]
    else:
        xb, yb = images, labels

    params = [p for p in model.parameters() if p.requires_grad]
    n = sum(p.numel() for p in params)
    if n == 0:
        if was_training: model.train()
        return 0.0

    # ---- forward with graph for second-order autodiff ----
    # Important: no torch.no_grad() here
    model.zero_grad(set_to_none=True)
    logits = model(xb)
    loss = criterion(logits, yb)

    # ∇ℓ with graph so we can differentiate it again
    grads = torch.autograd.grad(loss, params, create_graph=True, retain_graph=True)
    g_flat = torch.cat([gi.reshape(-1) for gi in grads])

    # init v ~ unit vector
    with torch.no_grad():
        v = torch.randn(n, device=g_flat.device)
        v /= (v.norm() + 1e-12)

    lam_prev = None
    for _ in range(iters):
        # H v = ∇[(∇ℓ)·v]
        gv = (g_flat * v).sum()
        Hv_parts = torch.autograd.grad(gv, params, retain_graph=True)
        Hv = torch.cat([h.reshape(-1) for h in Hv_parts])
        if damping != 0.0:
            Hv = Hv + damping * v

        with torch.no_grad():
            Hv_norm = Hv.norm()
            if Hv_norm == 0 or torch.isnan(Hv_norm):
                lam = 0.0
                break
            v = Hv / (Hv_norm + 1e-12)
            lam = torch.dot(v, Hv).item()

            if lam_prev is not None:
                if abs(lam - lam_prev) / (abs(lam_prev) + 1e-12) < tol:
                    break
            lam_prev = lam

    # cleanup and restore mode
    del grads, g_flat, logits, loss
    if was_training: model.train()
    return float(lam_prev if lam_prev is not None else lam)

In [7]:
def setup_output_files(output_dir="output"): 

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    metadata_path = os.path.join(output_dir, "metadata.csv")
    output_data_path = os.path.join(output_dir, "output.csv")

    if os.path.exists(metadata_path):
            metadata = pd.read_csv(metadata_path)
    else:
        metadata = pd.DataFrame({
            "model_id": pd.Series(dtype="int"),
            "model_type": pd.Series(dtype="str"),
            "optimizer": pd.Series(dtype="str"),
            "learning_rate": pd.Series(dtype="float"),
            "num_epochs": pd.Series(dtype="int"),
            "train_time": pd.Series(dtype="float")
        })

    if os.path.exists(output_data_path):
        output_data = pd.read_csv(output_data_path)
    else:
        output_data = pd.DataFrame({
            "model_id": pd.Series(dtype="int"),
            "loss": pd.Series(dtype="float"),
            "sharpness": pd.Series(dtype="float")
        })

    return metadata, output_data

In [8]:
def load_output_files(output_dir="output"):
    metadata_path = os.path.join(output_dir, "metadata.csv")
    output_data_path = os.path.join(output_dir, "output.csv")

    metadata = pd.read_csv(metadata_path)
    output_data = pd.read_csv(output_data_path)

    return metadata, output_data

In [9]:
def save_output_files(metadata, output_data, output_dir="output"):

    metadata_path = os.path.join(output_dir, "metadata.csv")
    output_data_path = os.path.join(output_dir, "output.csv")
    
    metadata.to_csv(metadata_path, index=False)
    output_data.to_csv(output_data_path, index=False)

In [10]:
def train_model(model, optimizer, criterion, learning_rate, num_epochs, images, labels, num_sharpness_computations=100):
    
    optimizer.param_groups[0]['lr'] = learning_rate

    model = model.to(device)
    images = images.to(device)
    labels = labels.to(device)

    train_losses = np.empty(num_epochs)
    sharps = np.full(num_epochs, np.nan)

    model.train()

    print(f"Model: {model.__class__.__name__}")
    print(f"Optimizer: {optimizer.__class__.__name__}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Number of Epochs: {num_epochs}")
    if model.__class__.__name__ == 'SGD': 
        print(f"Momentum: {optimizer.defaults['momentum']}")
    else:
        print(f"Momentum: None")

    start_time = time.time()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_losses[epoch] = loss.item()
        
        if (epoch + 1) % (num_epochs // num_sharpness_computations) == 0 or epoch == 0:
            sharpness = compute_sharpness(model, criterion, images, labels, iters=10, 
                                    subsample=256)
            sharps[epoch] = sharpness
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Sharpness: {sharpness}")
            
    end_time = time.time()

    metadata, output_data, = setup_output_files("output")
    model_id = metadata.shape[0] + 1
    
    metadata.loc[metadata.shape[0]] ={
        "model_id": model_id,
        "model_type": model.__class__.__name__,
        "optimizer": optimizer.__class__.__name__,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs,
        "train_time": end_time - start_time,
        "momentum" : optimizer.defaults['momentum'] if model.__class__.__name__ == 'SGD' else 0
    }

    output_data = pd.concat([output_data, pd.DataFrame({
        "model_id": np.ones_like(train_losses) * model_id,
        "loss": train_losses,
        "sharpness": sharps
    })], ignore_index=True)

    save_output_files(metadata, output_data, output_dir="output")
    print("")

In [33]:
input_size = np.prod(all_images.shape[1:])
hidden_layer_1_size = 200
hidden_layer_2_size = 200
model = FullyConnectedNet(input_size, hidden_layer_1_size, hidden_layer_2_size, NUM_LABELS)

learning_rates = [2/20, 2/50, 2/80, 2/110]
momentums = [0.6, 0.7, 0.8, 0.9]
num_epochs = 5000
num_sharpness_computations = 100
    
for momentum in momentums:
    for lr in learning_rates:
        train_model(model=model, 
                    optimizer=torch.optim.SGD(model.parameters(), momentum = momentum), 
                    criterion=nn.MSELoss(), 
                    num_epochs=num_epochs,
                    learning_rate=lr,
                    images=images, 
                    labels=labels,
                    num_sharpness_computations=num_sharpness_computations
        )

Model: FullyConnectedNet
Optimizer: SGD
Learning Rate: 0.1
Number of Epochs: 5000
Momentum: None
Epoch [1/5000], Loss: 0.1221, Sharpness: 15.711069107055664
Epoch [50/5000], Loss: 0.0776, Sharpness: 11.312751770019531
Epoch [100/5000], Loss: 0.0743, Sharpness: 11.772115707397461
Epoch [150/5000], Loss: 0.0719, Sharpness: 13.58896255493164
Epoch [200/5000], Loss: 0.0699, Sharpness: 12.72451114654541
Epoch [250/5000], Loss: 0.0679, Sharpness: 11.672883033752441
Epoch [300/5000], Loss: 0.0661, Sharpness: 15.431102752685547
Epoch [350/5000], Loss: 0.0642, Sharpness: 15.693780899047852
Epoch [400/5000], Loss: 0.0623, Sharpness: 16.218812942504883
Epoch [450/5000], Loss: 0.0603, Sharpness: 19.035930633544922
Epoch [500/5000], Loss: 0.0584, Sharpness: 22.969892501831055
Epoch [550/5000], Loss: 0.0564, Sharpness: 22.77707290649414
Epoch [600/5000], Loss: 0.0544, Sharpness: 27.613428115844727
Epoch [650/5000], Loss: 0.0524, Sharpness: 29.211742401123047
Epoch [700/5000], Loss: 0.0503, Sharpness

KeyboardInterrupt: 

In [51]:
def plot_output_data(metadata, output_data, num_rows, num_cols, title, model_ids = []):

    if len(model_ids) == 0:
        model_ids = metadata['model_id'].unique()

    specs = [[{"secondary_y": True} for _ in range(num_cols)] for _ in range(num_rows)]
    fig = make_subplots(num_rows, num_cols, specs=specs)

    for i, model_id in enumerate(model_ids):
        row = i // num_cols + 1
        col = i % num_cols + 1

        model_data = output_data[output_data['model_id'] == model_id]
        model_info = metadata[metadata['model_id'] == model_id].iloc[0]
        
        threshold = 2 / model_info['learning_rate']
        
        x_vals = np.arange(1, len(model_data)+1)
        loss = model_data['loss']
        sharpness = model_data['sharpness']

        fig.add_trace(
            go.Scatter(x=x_vals, y=loss, mode='lines', name='Loss',
                       line=dict(color='blue')),
            row=row, col=col, secondary_y=False
        )

        fig.add_trace(
            go.Scatter(x=x_vals, y=sharpness, mode='markers', name='Sharpness', 
                       marker=dict(size=4, color='red')),
            row=row, col=col, secondary_y=True
        )

        fig.add_hline(
            y=threshold,
            row=row, col=col,
            secondary_y=True,
            line=dict(color='black', dash='dot', width=1),
            opacity=1.0
        )

        fig.update_xaxes(title_text="Epoch", row=row, col=col)
        fig.update_yaxes(title_text="Loss", row=row, col=col, secondary_y=False)
        fig.update_yaxes(
            title_text="Sharpness",
            row = row,
            col = col,
            secondary_y=True,
            range=[0,max(sharpness.max(), threshold * 1.2)],
            showgrid=False,
            tickmode='array',
            tickvals=[threshold / 2, threshold],
            ticktext=["1/η", "2/η"],
            ticks='outside',
            ticklen=6,
            tickwidth=1,
        )

    fig.update_layout(title_text=title, showlegend=False, height = 1200, width = 600)

    fig.show()

metadata, output_data = load_output_files()
plot_output_data(metadata, output_data, 4, 1, title="Gradient Descent")